# Project: classification of reviews for an online store

The store needs a tool that will search for toxic comments left by users in the description of the store's products and send them for moderation.

**Project goal:** Train the model to classify comments into positive and negative.

We have a data set with markings about the toxicity of edits.

The target value of the quality metric F1 is not less than 0.75.

**Data Description**

The data is in the toxic_comments.csv file.
The text column contains the text of the comment, and toxic is the target attribute.

## Data preparation

In [ ]:
!pip install spacy -q

In [ ]:
import re
import pandas as pd
import spacy
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,  GridSearchCV

In [ ]:
df_comm = pd.read_csv("/datasets/toxic_comments.csv")

In [ ]:
RANDOM_STATE=12345

In [ ]:
df_comm.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [ ]:
df_comm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [ ]:
df_comm.duplicated().sum()

0

In [ ]:
df_comm.isna().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

The dataset consists of 159,292 rows, a column with message texts and a column with the markup of the target feature. There are no gaps or duplicates in the dataset.

Let's look at the ratio of classes of the target feature

In [ ]:
df_comm['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

There is a pronounced imbalance of the target attribute (there are almost 10 times more non-toxic comments than toxic ones), but we will not change the dataset in order to get more honest work of the models.

To prepare the data for training, we will remove unnecessary characters and spaces using regular expressions, tokenize and lemmatize the text. We will add the cleaned text to the dataset. For convenience, we will perform the preparation using the function.

In [ ]:
def clean_text(text):

    text = text.lower()
    regular = r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
    regular_url = r'(http\S+)|(www\S+)|([\w\d]+www\S+)|([\w\d]+http\S+)'
    text = re.sub(regular, '', text)
    text = re.sub(r'(\d+\s\d+)|(\d+)',' NUM ', text)
    text = re.sub(r'\s+', ' ', text)
    return text


nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def lemmatize_text(text):
    text = clean_text(text)
    doc = nlp(text)
    text = [token.lemma_ for token in doc]
    text = ' '.join(text)
    return text

tqdm.pandas()
df_comm['lemm_text'] = df_comm['text'].progress_apply(lemmatize_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [ ]:
df_comm.head()

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,1,D'aww! He matches this background colour I'm s...,0,d'aww ! he match this background colour I be s...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man I be really not try to edit war it be ...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more I can not make any real suggestion on i...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


The next step is to select the features on which we will train the models.
 We divide the dataset into training and test samples; when dividing, we will take into account the imbalance of classes.

In [ ]:
X=df_comm['lemm_text']
y=df_comm['toxic']

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, stratify=y, random_state=12345)

## Models learning

We will use the pipeline  contains DecisionTree, Logistic Regression, KNN. Then we carry out cross-validation to select hyperparameters of the models. We display the training results of all models in a table.

In [ ]:
pipe = Pipeline([
    ('vect', TfidfVectorizer()),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

In [ ]:
param_grid = [
    {
    "vect__ngram_range": ((1, 1), (1, 2))
    },
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__min_samples_leaf': range(3,6),
        'models__max_depth': range(3,6)
    },
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE)],
        'models__C': range(5,15),
        'models__penalty': ['l1', 'l2', 'elasticnet']
    } ,
    {
        'models': [KNeighborsClassifier(n_neighbors=300)]

    }
                 ]

In [ ]:
grid_search = GridSearchCV(
    pipe,
    param_grid=param_grid,
    cv=3,
    scoring='f1',
    n_jobs=-1
)

In [ ]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('models',
                                        DecisionTreeClassifier(random_state=12345))]),
             n_jobs=-1,
             param_grid=[{'vect__ngram_range': ((1, 1), (1, 2))},
                         {'models': [DecisionTreeClassifier(random_state=12345)],
                          'models__max_depth': range(3, 6),
                          'models__min_samples_leaf': range(3, 6)},
                         {'models': [LogisticRegression(C=13,
                                                        random_state=12345)],
                          'models__C': range(5, 15),
                          'models__penalty': ['l1', 'l2', 'elasticnet']},
                         {'models': [KNeighborsClassifier(n_neighbors=300)]}],
             scoring='f1')

In [ ]:
res = pd.DataFrame(grid_search.cv_results_)
res = res.drop (['std_fit_time','mean_score_time',
                 'std_score_time','params','split0_test_score','split1_test_score',
                 'split2_test_score','std_test_score'], axis=1)
res

,mean_fit_time,param_vect__ngram_range,param_models,param_models__max_depth,param_models__min_samples_leaf,param_models__C,param_models__penalty,mean_test_score,rank_test_score
0,133.197824,"(1, 1)",NaN,NaN,NaN,NaN,NaN,0.691579,11
1,604.631311,"(1, 2)",NaN,NaN,NaN,NaN,NaN,0.679340,12
2,4.764830,NaN,DecisionTreeClassifier(random_state=12345),3,3,NaN,NaN,0.427721,22
3,4.720769,NaN,DecisionTreeClassifier(random_state=12345),3,4,NaN,NaN,0.427966,21
4,4.699417,NaN,DecisionTreeClassifier(random_state=12345),3,5,NaN,NaN,0.428153,20
5,5.151276,NaN,DecisionTreeClassifier(random_state=12345),4,3,NaN,NaN,0.472220,18
6,5.111781,NaN,DecisionTreeClassifier(random_state=12345),4,4,NaN,NaN,0.472283,17
7,5.119505,NaN,DecisionTreeClassifier(random_state=12345),4,5,NaN,NaN,0.473542,16
8,5.577193,NaN,DecisionTreeClassifier(random_state=12345),5,3,NaN,NaN,0.502856,13
9,5.585274,NaN,DecisionTreeClassifier(random_state=12345),5,4,NaN,NaN,0.502666,15


In [ ]:
print('The best model and its parameters:\n\n', grid_search.best_estimator_)
print ('The metric of the best model on cross-validation:', grid_search.best_score_)

The best model and its parameters:

 Pipeline(steps=[('vect', TfidfVectorizer()),
                ('models', LogisticRegression(C=13, random_state=12345))])
The metric of the best model on cross-validation: 0.7812920327938802


In [ ]:
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred_test = grid_search.best_estimator_.predict(X_test)
f1_test = f1_score(y_test, y_pred_test)

print ("F1 on test set is:", f1_test)

F1 on test set is: 0.7850360370666971


The best model chosen during cross-validation is logistic regression with the regularization parameter capacity =13, regularization l2, the value of the F1 metric on cross-validation is 78.1%, on the test sample 78.5% (meets the requirements of the project).

## Conclusions

According to the goal of the project - to train the model to identify toxic comments on the store's website, the necessary libraries and dataset were downloaded, and an initial assessment of the data was carried out.

At the second stage, the texts of comments were prepared for training models - unnecessary characters and spaces were removed, tokenization and lemmatization of the text was carried out.


The lemmatized text is transferred to a pipeline that includes the TF-IDF vectorizer, 3 models (logistic regression, decision tree, nearest neighbors method).

Cross-validation was carried out to select the best model and its hyperparameters, logistic regression was chosen as the best model, the F1 metric on cross-validation was 78.1%. Predictions were obtained on the test sample; the F1 metric on the test sample was 78.5%.


Thus, logistic regression, given its fast performance and satisfactory prediction quality, can be used to select toxic comments.